In [1]:
import sys
import os
current_dir = os.getcwd().rsplit("/", 1)[0]
sys.path.append(current_dir)

from alpaca.data import StockHistoricalDataClient
from utils import Database, InsiderScraper, config, CloudStorage
from datetime import datetime, timedelta

scraper = InsiderScraper()
db = Database()
stock_client = StockHistoricalDataClient(config.ALPACA_KEY,  config.ALPACA_SECRET)

In [2]:
# Getting all tickers that have been traded and putting them in a list
q = """select
distinct ticker
from `open_insider.ticker_data`"""
rows = db.query(q)
tickers = []
for row in rows:
    ticker = row.values()[0]
    tickers.append(ticker)
    
# tickers

In [4]:
# Creating table with final stock prices for each year
for year in range(2016, 2025):
    # Getting last business date of the year
    if year == 2024:
        eoy_date = datetime.now().date() - timedelta(1)
    else:
        eoy_date = f"{year}-12-31"
        eoy_date = datetime.strptime(eoy_date, "%Y-%m-%d")
    while eoy_date.weekday() >= 5:  # 5 and 6 represent Saturday and Sunday
        eoy_date -= timedelta(days=1)

    ticker_data = scraper.get_stock_price(stock_client, tickers, eoy_date)

    table = f"{year}_ticker_data"
    print(f"Dropping and recreating {table}\n")
    db.drop_table(table)
    db.cache_new_stock(ticker_data, table)

Dropping and recreating 2016_ticker_data

Dropping and recreating 2017_ticker_data

Dropping and recreating 2018_ticker_data

Dropping and recreating 2019_ticker_data

Dropping and recreating 2020_ticker_data

Dropping and recreating 2021_ticker_data

Dropping and recreating 2022_ticker_data

Dropping and recreating 2023_ticker_data

Dropping and recreating 2024_ticker_data



In [7]:
# Getting ROI for each year
for year in range(2016, 2025):
  q = f"""
  with grouped as (
  select
    tr.insider_name,
    tr.ticker,
    sum(price*qty) / sum(qty) as avg_cost,
    avg(ti.close) as eoy_price
  from 
    `open_insider.trades` tr
  join
    `open_insider.{year}_ticker_data` ti
  on
    tr.ticker = ti.ticker
  where
    tr.trade_type = "P - Purchase" and
    extract(year from tr.filing_date) = {year}
  group by
    insider_name, ticker),
  return as (
  select
    *,
    ((eoy_price - avg_cost) / avg_cost) * 100 as roi
  from
    grouped
  where
    avg_cost != 0)

  select
    avg(roi)
  from 
    return
  """

  resp = db.query(q)

  for i in resp:
    print(f"{year} ROI: {i.values()[0]}")

  print()

2016 ROI: 91.71606623016294

2017 ROI: 76.1086587511019

2018 ROI: 63.459828256462224

2019 ROI: 427.6977803674069

2020 ROI: 109.6035223974755

2021 ROI: 306.68019967270317

2022 ROI: 39.17685335617192

2023 ROI: 103.58932257195752

2024 ROI: 19.93276257331308

